In [1]:
import numba
from numba import types, typed

import numpy as np

In [30]:
spec_connection = [('key', types.ListType(types.int32)),
                   ('mutate_rate', types.float64),
                   ('mean_weight_values', types.float64[:]),
                   ('std_weight_values', types.float64[:]),
                   ('mean_weight', types.float64),
                   ('std_weight', types.float64)]
@numba.jitclass(spec_connection)
class ConnectionNB:
    def __init__(key, mutate_rate, mean_weight_values, std_weight_values):
        self.key = key
        self.mutate_rate = mutate_rate
        self.mean_weight_values = mean_weight_values
        self.std_weight_values = std_weight_values
        self.mean_weight = 0.0
        self.std_weight = 0.0
    
        self._mutate_mean()
        self._mutate_std()
        
    def get_mean_weight(self):
        return self.mean_bias
    
    def get_std_bias(self):
        return self.std_bias
    
    def get_key(self):
        return self.key
    
    def mutate_mean(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self._mutate_mean()
    
    def _mutate_mean(self):
        self.mean_weight = np.random.choice(self.mean_weight_values, size=1)[0]
        
    def mutate_std(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self._mutate_std()
            
    def _mutate_std(self):
        self.std_weight = np.random.choice(self.std_weight_values, size=1)[0]

In [24]:
spec_node = [('key', types.int64),
             ('mutate_rate', types.float64),
             ('mean_bias_values', types.float64[:]),
             ('std_bias_values', types.float64[:]),
             ('mean_bias', types.float64),
             ('std_bias', types.float64)]

@numba.jitclass(spec_node)
class NodeNB:
    
    def __init__(self, key, mutate_rate, mean_bias_values, std_bias_values):
        self.key = key
        self.mutate_rate = mutate_rate
        self.mean_bias_values = mean_bias_values
        self.std_bias_values = std_bias_values
        self.mean_bias = 0.0
        self.std_bias = 0.0
        
        self._mutate_mean()
        self._mutate_std()
        
    def get_mean_bias(self):
        return self.mean_bias
    
    def get_std_bias(self):
        return self.std_bias
    
    def get_key(self):
        return self.key
    
    def mutate_mean(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self._mutate_mean()
    
    def _mutate_mean(self):
        self.mean_bias = np.random.choice(self.mean_bias_values, size=1)[0]
    
    def mutate_std(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self._mutate_std()
            
    def _mutate_std(self):
        self.std_bias = np.random.choice(self.std_bias_values, size=1)[0]

In [25]:
nodes_dict_type = (types.int32, NodeNB)
connections_dict_type = (types.ListType(types.int32), ConnectionNB)

# A container class with:
# * member 'd' holding a typed dictionary of int64 -> unicode string (kv_ty)
# * member 'l' holding a typed list of float64
spec_genome = [('key', types.int32),
               ('n_input', types.int16),
               ('n_output', types.int16),
               ('node_genes', types.DictType(*nodes_dict_type)),
               ('connection_genes', types.DictType(*connections_dict_type)),
               ('fitness', types.float64)]



@numba.jitclass(spec_genome)
class GenomeNB:
    def __init__(self, key, n_input, n_output):
        self.key = key
        self.n_input = n_input
        self.n_output = n_output
        
        self.node_genes = typed.Dict.empty(*nodes_dict_type)
        self.connection_genes = typed.Dict.empty(*connections_dict_type)
        
        self.fitness = 0
    
    def get_key(self):
        return self.key
    
    def set_node_genes(self, node_genes):
        self.node_genes = node_genes
    
    def get_node_genes(self):
        return self.node_genes
    
    def set_connection_genes(self, connection_genes):
        self.connection_genes = connection_genes
    
    def get_connection_genes(self):
        return self.connection_genes
    
    def set_fitness(self, fitness):
        self.fitness = fitness
        
    def get_fitness(self):
        return self.fitness

In [31]:
@numba.njit()
def get_numba_array(list_):
    return np.array(list_)


@numba.njit(debug=True)
def initialize_nodes(n_nodes, mutate_rate, mean_bias_values, std_bias_values):
    nodes = {}
    for i in range(n_nodes):
        node = NodeNB(i, 
                      mutate_rate, 
                      mean_bias_values, 
                      std_bias_values)

        nodes[i] = node
    return nodes


@numba.njit(debug=True)
def initialize_connections(n_connections, mutate_rate, mean_weight_values, std_weight_values):
    connections = {}
    for i in range(n_connections):
        connection = ConnectionNB([i, i-1], 
                      mutate_rate, 
                      mean_weight_values, 
                      std_weight_values)

        connections[i] = connection
    return connections

## Comparison

In [27]:
n_nodes = 5000000
n_connections = 30000
mutate_rate = 0.5

mean_bias_values = [-0.2, 0.0, 0.2]
std_bias_values = [0.05, 0.1, 0.5]


typed_mean_bias_values = get_numba_array(mean_bias_values)
typed_std_bias_values = get_numba_array(std_bias_values)


mean_weight_values = [-0.2, 0.0, 0.2]
std_weight_values = [0.05, 0.1, 0.5]

typed_mean_weight_values = get_numba_array(mean_weight_values)
typed_std_weight_values = get_numba_array(std_weight_values)


/home/alberto/.local/share/virtualenvs/Neuro-Evolution-BNN-_WcBvtZA/lib/python3.6/site-packages/numba/ir_utils.py:1969: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'list_' of function 'get_numba_array'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-26-11b13204c307>", line 2:
@numba.njit()
def get_numba_array(list_):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [19]:
%%time
nodes = initialize_nodes(n_nodes, mutate_rate, typed_mean_bias_values, typed_std_bias_values)

CPU times: user 2.85 s, sys: 233 ms, total: 3.09 s
Wall time: 4.35 s


In [32]:
connections = initialize_connections(n_connections, mutate_rate, typed_mean_bias_values, typed_std_bias_values)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.typeinfer.CallConstraint object at 0x7f022ee11828>.
too many positional arguments
[1] During: resolving callee type: jitclass.ConnectionNB#7f022ee2b8d0<key:ListType[int32],mutate_rate:float64,mean_weight_values:array(float64, 1d, A),std_weight_values:array(float64, 1d, A),mean_weight:float64,std_weight:float64>
[2] During: typing of call at <ipython-input-31-11b13204c307> (26)

Enable logging at debug level for details.

File "<ipython-input-31-11b13204c307>", line 26:
def initialize_connections(n_connections, mutate_rate, mean_weight_values, std_weight_values):
    <source elided>
                      mean_weight_values, 
                      std_weight_values)
                      ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of 

In [131]:
# for i in range(100):
#     print(nodes[i].get_key())
#     print(nodes[i].get_mean_bias())

In [132]:
%%time
nodes_py = initialize_nodes_simple(n_nodes, mutate_rate, typed_mean_bias_values, typed_std_bias_values)

CPU times: user 1min 38s, sys: 909 ms, total: 1min 39s
Wall time: 1min 43s
